###  Day 6: Memory Reallocation

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

- The banks start with 0, 2, 7, and 0 blocks. The third bank has the most blocks, so it is chosen for redistribution.
- Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
- Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
- Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
- The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
- The third bank is chosen, and the same thing happens: 2 4 1 2.
- At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

### Part One

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

### Part Two

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

In [15]:
# sample = True
sample = False
path = 'Inputs\\day_06.txt' if sample == False else 'Inputs\\day_06_sample.txt'  
with open(path) as f: puz = [int(i) for i in f.read().split('\t')]

bank_status = puz

num_of_banks = len(bank_status)
bank_status_list = [] # <- a list of the block status across the memory banks at the end of each cycle
bank_status_list.append(bank_status.copy())
cycles = 0
first_duple_cycle  = 0

while cycles < 10000000: # <- could do something with if count    
    
    cycles += 1

    # identify block with maximum blocks, 
    # and split the blocks to distribuite into evenly distributable and the reaminder
    max_block_idx = bank_status.index(max(bank_status))
    
    blocks_to_spread_evenly = bank_status[max_block_idx] // num_of_banks
    remainder_blocks        = bank_status[max_block_idx] %  num_of_banks

    # remove blocks from the max block index
    bank_status[max_block_idx] = 0
    
    for idx, memory_bank in enumerate(bank_status):
        bank_status[idx] += blocks_to_spread_evenly 
    
    # add the remainder to all relevant indexes
    adding_idx = max_block_idx+1
    while remainder_blocks > 0:
        if adding_idx > num_of_banks-1: adding_idx = 0

        bank_status[adding_idx] += 1
        remainder_blocks -= 1
        adding_idx += 1    
    
    # note the cycle point of the first duplicate
    if bank_status_list.count(bank_status) > 0 and first_duple_cycle == 0:
        first_duple_cycle = cycles
        
    # if hitting a duplicate for the second time, stop
    if bank_status_list.count(bank_status) > 1:
        break
        
    bank_status_list.append(bank_status.copy())
    

print(f'First duplicate cycle = {first_duple_cycle}')  
print(f'Infinite loop size    = {cycles - first_duple_cycle}')  


First duplicate cycle = 12841
Infinite loop size    = 8038


### dev code with commentary

In [76]:
# sample = True
sample = False
path = 'Inputs\\day_06.txt' if sample == False else 'Inputs\\day_06_sample.txt'  
with open(path) as f: puz = [int(i) for i in f.read().split('\t')]

block_status = puz

num_of_banks = len(block_status)
cycles = 0
block_status_list = []
block_status_list.append(block_status.copy())
first_duple_cycle = 0

while cycles < 1000000:
#     print(f'starting_block_status = {block_status}')

    cycles += 1

    # FIND BLOCK WITH LARGEST NUMBER (LOWEST INDEX FOR TIES), SAVE NUM TO 'BLOCKS TO DISTRIBUTE' AND NOTE 'BLOCK INDEX'
    block_index = block_status.index(max(block_status))
    blocks_to_distribute = block_status[block_index]
    
#     print(f'block_index = {block_index}\nblocks_to_distribute = {blocks_to_distribute}')

    # SET VALUE OF 'BLOCK INDEX' 0
    block_status[block_index] = 0
#     print(f'block_index {block_index} set to {block_status[block_index]}')

    # GET THE 'MODULUS' FROM THE 'BLOCKS TO DISTRIBUTE' VALUE
    remainder_blocks = blocks_to_distribute % len(block_status)
#     print(f'remainder_blocks = {remainder_blocks}')
    
    # DIVIDE THE 'NUMBER OF COLUMNS' (LEN(INPUT)) BY THE (BLOCKS TO DISTRIBUTE' - REMAINDER') TO GET THE 'EVEN_SPREAD'
    blocks_to_spread_evenly = int((blocks_to_distribute-remainder_blocks) / len(block_status))
#     print(f'blocks_to_spread_evenly = {blocks_to_spread_evenly}')
    
    # ADD 'EVEN_SPREAD' TO ALL BLOCKS
    for idx, memory_bank in enumerate(block_status):
        block_status[idx] += blocks_to_spread_evenly 
    
#     print(f'block_status = {block_status}')
    
    adding_idx = block_index+1

    while remainder_blocks > 0:
        if adding_idx > num_of_banks-1: adding_idx = 0

        block_status[adding_idx] += 1
        remainder_blocks -= 1
        adding_idx += 1

#     print(f'closing_block_status = {block_status}')       
    
    # note the cycle point of the first duplicate
    if block_status_list.count(block_status) > 0 and first_duple_cycle == 0:
        first_duple_cycle = cycles
        
    # if hitting a duplicate for the second time, stop
    if block_status_list.count(block_status) > 1:
        break
    else:
        block_status_list.append(block_status.copy())

#     print(f'block_status_list = {block_status_list}\n')  

print(f'First duplicate cycle count = {first_duple_cycle}')  
print(f'Infinite loop size          = {cycles - first_duple_cycle}')  


        # ADD FINAL 'BLOCK STATUS' TO LIST 
        # IF COUNT OF BLOCK STATUS IN 'BLOCK STATUS LIST' > 1 THEN STOP
 



First duplicate cycle count = 12841
Infinite loop size          = 8038
